In [12]:
import sqlite3
import zipfile
from io import BytesIO

import numpy as np
import pandas as pd
import requests


In [13]:
DB_PATH = "challenge.db"  # change if needed
conn = sqlite3.connect(DB_PATH)


In [14]:
access_df = pd.read_sql_query("""
    SELECT
        FIPS,
        State,
        County,
        PCT_LACCESS_POP15,
        PCT_LACCESS_LOWI15,
        PCT_LACCESS_SNAP15,
        PCT_LACCESS_HHNV15
    FROM access
""", conn)

access_df["FIPS"] = access_df["FIPS"].astype(str).str.zfill(5)
access_df.head()


,FIPS,State,County,PCT_LACCESS_POP15,PCT_LACCESS_LOWI15,PCT_LACCESS_SNAP15,PCT_LACCESS_HHNV15
0,01001,AL,Autauga,32.062255,11.991125,4.608749,3.351332
1,01003,AL,Baldwin,16.767489,5.424427,1.298900,1.905114
2,01005,AL,Barbour,22.105560,10.739667,4.303147,4.329378
3,01007,AL,Bibb,4.230324,2.601627,0.676710,2.821427
4,01009,AL,Blount,6.497380,2.880150,0.812727,3.336414


In [15]:
cities_df = pd.read_sql_query("""
    SELECT
        TractFIPS,
        Population2010,
        DIABETES_CrudePrev,
        OBESITY_CrudePrev,
        LPA_CrudePrev,
        PHLTH_CrudePrev,
        MHLTH_CrudePrev
    FROM five_hundred_cities
""", conn)

cities_df["FIPS"] = cities_df["TractFIPS"].astype(str).str.slice(0, 5)

# Drop rows missing population or key outcomes
cities_df = cities_df.dropna(subset=["Population2010", "DIABETES_CrudePrev", "OBESITY_CrudePrev", "LPA_CrudePrev"])

def wavg(g: pd.DataFrame, col: str) -> float:
    v = g[col].astype(float)
    w = g["Population2010"].astype(float)
    if w.sum() <= 0:
        return np.nan
    return float(np.average(v, weights=w))

county_health_df = (
    cities_df.groupby("FIPS", as_index=False)
    .apply(lambda g: pd.Series({
        "diabetes_rate": wavg(g, "DIABETES_CrudePrev"),
        "obesity_rate": wavg(g, "OBESITY_CrudePrev"),
        "inactivity_rate": wavg(g, "LPA_CrudePrev"),
        "poor_physical_health": wavg(g, "PHLTH_CrudePrev") if g["PHLTH_CrudePrev"].notna().any() else np.nan,
        "poor_mental_health": wavg(g, "MHLTH_CrudePrev") if g["MHLTH_CrudePrev"].notna().any() else np.nan,
        "pop2010": float(g["Population2010"].sum()),
    }))
    .reset_index(drop=True)
)

county_health_df["FIPS"] = county_health_df["FIPS"].astype(str).str.zfill(5)
county_health_df.head()


,FIPS,diabetes_rate,obesity_rate,inactivity_rate,poor_physical_health,poor_mental_health,pop2010
0,10003,14.232654,39.848538,38.391434,16.113002,17.189678,70851.0
1,10730,14.731505,38.806230,35.352871,15.883488,16.010577,268987.0
2,10830,7.797041,31.716174,25.654043,10.486391,13.659566,1521.0
3,10890,11.909898,35.672965,30.702233,14.558571,15.120165,178484.0
4,10970,14.330603,37.866306,36.099728,16.105554,16.381763,195045.0


In [16]:
county_df = county_health_df.merge(access_df, on="FIPS", how="inner")
print("county_df shape:", county_df.shape)
county_df.head()


county_df shape: (274, 13)


,FIPS,diabetes_rate,obesity_rate,inactivity_rate,poor_physical_health,poor_mental_health,pop2010,State,County,PCT_LACCESS_POP15,PCT_LACCESS_LOWI15,PCT_LACCESS_SNAP15,PCT_LACCESS_HHNV15
0,10003,14.232654,39.848538,38.391434,16.113002,17.189678,70851.0,DE,New Castle,22.686807,4.750103,1.950658,1.091094
1,11001,8.712104,26.150140,24.344025,9.334466,11.624444,601690.0,DC,District of Columbia,2.108658,0.785898,0.400502,0.460118
2,12001,7.428068,29.013784,26.451319,11.484560,16.494655,124343.0,FL,Alachua,15.227937,6.427962,1.618978,2.064161
3,12009,11.710821,31.840647,32.321369,15.360297,14.900320,179156.0,FL,Brevard,40.248706,12.312836,4.128732,1.942404
4,12011,10.471150,26.826611,30.162411,13.495619,13.496387,1207224.0,FL,Broward,9.066691,2.476416,0.787433,0.428416


In [18]:
YEAR = 2010  # choose 2021, 2022, 2023, 2024 depending on availability on BLS site

qcew_zip_url = f"https://data.bls.gov/cew/data/files/{YEAR}/csv/{YEAR}_annual_by_area.zip"
resp = requests.get(qcew_zip_url, timeout=120)
resp.raise_for_status()

z = zipfile.ZipFile(BytesIO(resp.content))
[zname for zname in z.namelist() if zname.lower().endswith(".csv")][:10]


['2010.annual.by_area/2010.annual 01000 Alabama -- Statewide.csv',
 '2010.annual.by_area/2010.annual 01001 Autauga County, Alabama.csv',
 '2010.annual.by_area/2010.annual 01003 Baldwin County, Alabama.csv',
 '2010.annual.by_area/2010.annual 01005 Barbour County, Alabama.csv',
 '2010.annual.by_area/2010.annual 01007 Bibb County, Alabama.csv',
 '2010.annual.by_area/2010.annual 01009 Blount County, Alabama.csv',
 '2010.annual.by_area/2010.annual 01011 Bullock County, Alabama.csv',
 '2010.annual.by_area/2010.annual 01013 Butler County, Alabama.csv',
 '2010.annual.by_area/2010.annual 01015 Calhoun County, Alabama.csv',
 '2010.annual.by_area/2010.annual 01017 Chambers County, Alabama.csv']

In [19]:
# Find the first CSV inside the zip
csv_name = next(n for n in z.namelist() if n.lower().endswith(".csv"))

qcew = pd.read_csv(
    z.open(csv_name),
    dtype={"area_fips": str, "industry_code": str},
    low_memory=False
)

print("QCEW columns:", list(qcew.columns))
qcew.head()


QCEW columns: ['area_fips', 'own_code', 'industry_code', 'agglvl_code', 'size_code', 'year', 'qtr', 'disclosure_code', 'area_title', 'own_title', 'industry_title', 'agglvl_title', 'size_title', 'annual_avg_estabs_count', 'annual_avg_emplvl', 'total_annual_wages', 'taxable_annual_wages', 'annual_contributions', 'annual_avg_wkly_wage', 'avg_annual_pay', 'lq_disclosure_code', 'lq_annual_avg_estabs_count', 'lq_annual_avg_emplvl', 'lq_total_annual_wages', 'lq_taxable_annual_wages', 'lq_annual_contributions', 'lq_annual_avg_wkly_wage', 'lq_avg_annual_pay', 'oty_disclosure_code', 'oty_annual_avg_estabs_count_chg', 'oty_annual_avg_estabs_count_pct_chg', 'oty_annual_avg_emplvl_chg', 'oty_annual_avg_emplvl_pct_chg', 'oty_total_annual_wages_chg', 'oty_total_annual_wages_pct_chg', 'oty_taxable_annual_wages_chg', 'oty_taxable_annual_wages_pct_chg', 'oty_annual_contributions_chg', 'oty_annual_contributions_pct_chg', 'oty_annual_avg_wkly_wage_chg', 'oty_annual_avg_wkly_wage_pct_chg', 'oty_avg_annual_

,area_fips,own_code,industry_code,agglvl_code,size_code,year,qtr,disclosure_code,area_title,own_title,...,oty_total_annual_wages_chg,oty_total_annual_wages_pct_chg,oty_taxable_annual_wages_chg,oty_taxable_annual_wages_pct_chg,oty_annual_contributions_chg,oty_annual_contributions_pct_chg,oty_annual_avg_wkly_wage_chg,oty_annual_avg_wkly_wage_pct_chg,oty_avg_annual_pay_chg,oty_avg_annual_pay_pct_chg
0,01000,0,10,50,0,2010,A,NaN,Alabama -- Statewide,Total Covered,...,929031263,1.3,96796711,0.7,231612899,118.3,17,2.2,867,2.2
1,01000,1,10,51,0,2010,A,NaN,Alabama -- Statewide,Federal Government,...,326781981,8.6,0,0.0,0,0.0,6,0.4,321,0.5
2,01000,1,102,52,0,2010,A,NaN,Alabama -- Statewide,Federal Government,...,326781981,8.6,0,0.0,0,0.0,6,0.4,321,0.5
3,01000,1,1021,53,0,2010,A,NaN,Alabama -- Statewide,Federal Government,...,35677247,5.4,0,0.0,0,0.0,91,8.9,4728,8.9
4,01000,1,1022,53,0,2010,A,NaN,Alabama -- Statewide,Federal Government,...,44702,10.6,0,0.0,0,0.0,-6,-0.8,-320,-0.8


In [20]:
# Keep county-only rows: 5-digit FIPS
qcew_county = qcew[qcew["area_fips"].str.fullmatch(r"\d{5}")].copy()

# Sanity check expected annual avg employment column
if "annual_avg_emplvl" not in qcew_county.columns:
    raise RuntimeError(
        "annual_avg_emplvl not found. You may have downloaded a quarterly file by mistake "
        "or BLS changed the schema. Columns were: "
        + ", ".join(qcew_county.columns[:30])
    )

qcew_county[["area_fips", "industry_code", "annual_avg_emplvl"]].head()


,area_fips,industry_code,annual_avg_emplvl
0,01000,10,1813155
1,01000,10,58395
2,01000,102,58395
3,01000,1021,11976
4,01000,1022,12


In [29]:
# Normalize industry codes: turn "31-33" into "31_33" etc.
qcew_county["industry_code_norm"] = qcew_county["industry_code"].str.replace("-", "_", regex=False)

# Choose sectors (edit this list)
SECTORS = {
    "31_33": "Manufacturing",
    "48_49": "Transportation and Warehousing",
    "62": "Health Care and Social Assistance",
    "44_45": "Retail Trade",
    "72": "Accommodation and Food Services",
    "23": "Construction",
    "92": "Public Administration",
    "54": "Professional, Scientific, and Technical Services",
    "52": "Finance and Insurance",
}

# We also need total employment. In many QCEW files, total is industry_code 10 or 10_?
# We'll compute total as sum of all sectors present in that county if a "10" total isn't present.

q = qcew_county.copy()
q["annual_avg_emplvl"] = pd.to_numeric(q["annual_avg_emplvl"], errors="coerce")

# Pivot selected sectors
sector_rows = q[q["industry_code_norm"].isin(SECTORS.keys())].copy()

sector_piv = sector_rows.pivot_table(
    index="area_fips",
    columns="industry_code_norm",
    values="annual_avg_emplvl",
    aggfunc="sum"
).reset_index().rename(columns={"area_fips": "FIPS"})

sector_piv["FIPS"] = sector_piv["FIPS"].astype(str).str.zfill(5)

# Compute total employment per county from all industries in file (more robust than relying on code "10")
total_emp = (
    q.groupby("area_fips", as_index=False)["annual_avg_emplvl"]
    .sum()
    .rename(columns={"area_fips": "FIPS", "annual_avg_emplvl": "emp_total"})
)
total_emp["FIPS"] = total_emp["FIPS"].astype(str).str.zfill(5)

sector_emp = sector_piv.merge(total_emp, on="FIPS", how="left")

# Convert to shares (% of total employment)
for code in SECTORS.keys():
    if code in sector_emp.columns:
        sector_emp[f"share_{code}"] = (sector_emp[code] / sector_emp["emp_total"]) * 100.0

# Keep only the share columns
share_cols = ["FIPS"] + [f"share_{code}" for code in SECTORS.keys() if f"share_{code}" in sector_emp.columns]
sector_share = sector_emp[share_cols].copy()

sector_share.head()


,FIPS


In [30]:
# --- FIXED QCEW SECTOR SHARE BLOCK (county-only, correct denominator) ---

import numpy as np
import pandas as pd

# Normalize industry codes: turn "31-33" into "31_33" etc.
qcew_county = qcew_county.copy()
qcew_county["area_fips"] = qcew_county["area_fips"].astype(str).str.zfill(5)
qcew_county["industry_code_norm"] = qcew_county["industry_code"].astype(str).str.replace("-", "_", regex=False)
qcew_county["annual_avg_emplvl"] = pd.to_numeric(qcew_county["annual_avg_emplvl"], errors="coerce")

# Keep only true counties (exclude aggregates like 01000)
qcew_county = qcew_county[
    qcew_county["area_fips"].str.fullmatch(r"\d{5}") &
    (~qcew_county["area_fips"].str.endswith("000"))
].dropna(subset=["annual_avg_emplvl"]).copy()

# Choose sectors (edit this list)
SECTORS = {
    "31_33": "Manufacturing",
    "48_49": "Transportation and Warehousing",
    "62": "Health Care and Social Assistance",
    "44_45": "Retail Trade",
    "72": "Accommodation and Food Services",
    "23": "Construction",
    "92": "Public Administration",
    "54": "Professional, Scientific, and Technical Services",
    "52": "Finance and Insurance",
}

q = qcew_county

# --- Correct total employment per county: use NAICS "10" (All industries total) ---
totals = q[q["industry_code_norm"] == "10"][["area_fips", "annual_avg_emplvl"]].copy()
totals = totals.rename(columns={"area_fips": "FIPS", "annual_avg_emplvl": "emp_total"})

if totals.empty:
    raise RuntimeError(
        "Could not find total employment rows (industry_code == '10'). "
        "Run: q['industry_code_norm'].value_counts().head(30) and check what code represents total."
    )

# Pivot selected sectors (absolute employment levels)
sector_rows = q[q["industry_code_norm"].isin(SECTORS.keys())][
    ["area_fips", "industry_code_norm", "annual_avg_emplvl"]
].copy()

sector_piv = sector_rows.pivot_table(
    index="area_fips",
    columns="industry_code_norm",
    values="annual_avg_emplvl",
    aggfunc="sum"
).reset_index().rename(columns={"area_fips": "FIPS"})

# Merge with totals (inner => keep only counties with a valid total)
sector_emp = sector_piv.merge(totals, on="FIPS", how="inner")

# Convert to shares (% of total employment)
for code in SECTORS.keys():
    if code in sector_emp.columns:
        sector_emp[f"share_{code}"] = (sector_emp[code] / sector_emp["emp_total"]) * 100.0

# Keep only shares (plus FIPS)
share_cols = ["FIPS"] + [f"share_{code}" for code in SECTORS.keys() if f"share_{code}" in sector_emp.columns]
sector_share = sector_emp[share_cols].copy()

# Optional sanity checks
# 1) make sure no "xxx00" aggregates slipped in
assert not sector_share["FIPS"].str.endswith("000").any()

# 2) shares should be plausible (not all tiny); total share across chosen sectors will be < 100 (we chose a subset)
sector_share.head()


RuntimeError: Could not find total employment rows (industry_code == '10'). Run: q['industry_code_norm'].value_counts().head(30) and check what code represents total.